# 🎬 Academy Awards Analysis
## Investigating Trends in Oscar-Winning Movies
### Author: Judd Jacobs

This project analyzes historical **Academy Award-winning films** using data from **Wikipedia** (scraped via `pandas.read_html()`) and the **Kaggle Oscar Awards dataset**. The analysis explores:
- 🏆 **Best Picture trends by genre**
- 🎭 **Box office revenue & IMDb ratings**
- 📈 **Long-term trends in Oscar-winning films**
- ☁️ **Stretch Goal**: **Word Cloud analysis of Wikipedia movie summaries**

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Set plotting style
sns.set_style("whitegrid")

# Ensure necessary NLTK components are downloaded
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juddjacobs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juddjacobs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 🗂 Data Collection: Scraping Wikipedia
We will extract **Best Picture winners** and relevant metadata using:
- **`pandas.read_html()`** to extract the table structure.
- **`BeautifulSoup`** to identify "winning" rows based on background color.

In [2]:
# Wikipedia URL for Best Picture winners
wiki_url = "https://en.wikipedia.org/wiki/List_of_Academy_Award%E2%80%93winning_films"

# Fetch page content
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, "html.parser")

# Use pandas to extract the table
tables = pd.read_html(wiki_url)

# Select the correct table
best_picture_df = tables[0]

# Print the first few rows
best_picture_df.head()


,Film,Year,Awards,Nominations
0,Anora,2024,5,6
1,The Brutalist,2024,3,10
2,Emilia Pérez,2024,2,13
3,Wicked,2024,2,10
4,Dune: Part Two,2024,2,5


In [5]:
# Find the Wikipedia table with BeautifulSoup
wikipedia_table = soup.find_all("table", {"class": "wikitable"})[0]  # Adjust index if needed

# Extract all rows
rows = wikipedia_table.find_all("tr")

# List to store "Winner" status
winning_status = []

# Loop through rows and check for background color "#EEDD82"
for row in rows[1:]:  # Skip header row
    style = row.get("style", "")
    
    # Check if the row has the background color for winners
    if "background:#EEDD82" in style.replace(" ", ""):  # Remove spaces for consistency
        winning_status.append("Winner")
    else:
        winning_status.append("Nominee")

# Ensure the list length matches the DataFrame
if len(winning_status) == len(best_picture_df):
    best_picture_df["Status"] = winning_status

# Display updated DataFrame
best_picture_df.head()


,Film,Year,Awards,Nominations,Status
0,Anora,2024,5,6,Winner
1,The Brutalist,2024,3,10,Nominee
2,Emilia Pérez,2024,2,13,Nominee
3,Wicked,2024,2,10,Nominee
4,Dune: Part Two,2024,2,5,Nominee


In [7]:
# Connect to (or create) database
conn = sqlite3.connect("academy_awards.db")

# Save to SQLite table
best_picture_df.to_sql("best_picture_winners", conn, if_exists="replace", index=False)
print("Data saved to SQLite database!")


Data saved to SQLite database!


## 📥 Data Collection: Kaggle Dataset
We will load additional Oscar award data from a structured Kaggle dataset.

In [ ]:
# Load Kaggle dataset
kaggle_df = pd.read_csv("References/oscars.csv")

# Display dataset structure
kaggle_df.head()

## 🛠 Data Cleaning & Storage
We will clean and merge Wikipedia & Kaggle data, then store it in an **SQLite database**.

In [ ]:
# Clean Wikipedia data
movies_df["Year"] = movies_df["Year"].str.extract(r"(\d{4})").astype(float)

# Merge Wikipedia and Kaggle data
merged_df = pd.merge(movies_df, kaggle_df, left_on="Best Picture Winner", right_on="Film", how="left")

# Save to SQLite database
conn = sqlite3.connect("academy_awards.db")
merged_df.to_sql("best_picture_winners", conn, if_exists="replace", index=False)

print("Data successfully stored in SQLite database!")

## 📊 Exploratory Data Analysis
We will explore trends in **Best Picture winners** by genre and other relevant statistics.

In [ ]:
# Check unique genres in Kaggle dataset
print(kaggle_df["Genre"].unique())

# Count of Best Picture winners by genre
genre_counts = kaggle_df["Genre"].value_counts()

# Plot the genre distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=genre_counts.index, y=genre_counts.values, palette="Blues_r")
plt.xticks(rotation=45)
plt.xlabel("Genre")
plt.ylabel("Number of Wins")
plt.title("Best Picture Wins by Genre")
plt.show()

## 💰 Box Office & IMDb Ratings
We will analyze **box office revenue** and IMDb ratings of Best Picture winners.

In [ ]:
# Scatter plot: Box Office Revenue vs IMDb Ratings
plt.figure(figsize=(10, 6))
sns.scatterplot(x=kaggle_df["BoxOffice"], y=kaggle_df["IMDb Rating"], hue=kaggle_df["Year"], palette="coolwarm")
plt.xlabel("Box Office Revenue (in millions)")
plt.ylabel("IMDb Rating")
plt.title("Box Office Revenue vs IMDb Ratings for Best Picture Winners")
plt.show()

## ☁️ Stretch Goal: Word Cloud (Wikipedia Movie Summaries)
If Wikipedia summaries are accessible, generate a **word cloud** from commonly used words in movie descriptions.

In [ ]:
# Sample Wikipedia summary text (replace with actual summaries if available)
sample_text = "This is a sample summary of a Best Picture-winning film. It tells the story of love, ambition, and success."

# Tokenize & remove stopwords
tokens = word_tokenize(sample_text.lower())
filtered_words = [word for word in tokens if word.isalnum() and word not in stopwords.words("english")]

# Generate Word Cloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(filtered_words))

# Display Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Wikipedia Movie Summaries")
plt.show()

In [ ]:
oscars = pd.read_csv("data/oscars.csv", sep='\t', on_bad_lines='skip')
oscars = oscars.dropna()
oscars = oscars.drop_duplicates()
oscars = oscars.reset_index(drop=True)

# Step 3: Data Cleaning

In [ ]:
# Clean movie metadata
def clean_movie_data(df):
    """
    Handle missing values and standardize column names in the movie dataset.
    """
    df.dropna(subset=["title", "release_year"], inplace=True)
    df.fillna({"box_office": 0, "runtime": df["runtime"].median()}, inplace=True)
    return df

# Clean speech transcripts (stretch goal)
# def preprocess_speech_text(text):
#     """
#     Tokenize and clean Oscar acceptance speech text for word frequency analysis.
#     """
#     nltk.download("stopwords")
#     nltk.download("punkt")
#     tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
#     filtered_words = [word for word in tokens if word.isalnum() and word not in stopwords.words("english")]
#     return " ".join(filtered_words)

# speech_df["cleaned_speech"] = speech_df["speech_text"].apply(preprocess_speech_text)



In [ ]:
# Step 4: Store Data in SQLite Database
conn = sqlite3.connect("academy_awards.db")
awards_df.to_sql("awards", conn, if_exists="replace", index=False)
speech_df.to_sql("speeches", conn, if_exists="replace", index=False)



In [ ]:
# Step 5: SQL Queries & Analysis
## Query genres of Best Picture winners over decades
query = """
SELECT genre, COUNT(*) AS num_wins, strftime('%Y', award_year) AS decade
FROM awards
WHERE category = 'Best Picture'
GROUP BY genre, decade
ORDER BY decade ASC;
"""
genre_trends_df = pd.read_sql(query, conn)

## Query word frequency in acceptance speeches
query = """
SELECT cleaned_speech FROM speeches;
"""
speech_texts = pd.read_sql(query, conn)



In [ ]:
# Step 6: Data Visualization
## Bar Chart - Best Picture Wins by Genre
plt.figure(figsize=(12,6))
sns.barplot(x="genre", y="num_wins", hue="decade", data=genre_trends_df)
plt.xticks(rotation=45)
plt.title("Best Picture Wins by Genre Over Decades")
plt.show()

## Scatter Plot - Box Office vs IMDb Ratings
plt.figure(figsize=(10,5))
sns.scatterplot(x="box_office", y="imdb_rating", hue="decade", data=awards_df)
plt.title("Box Office Revenue vs IMDb Ratings for Oscar Winners")
plt.show()

## Word Cloud - Common Words in Acceptance Speeches
all_text = " ".join(speech_texts["cleaned_speech"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_text)
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Most Common Words in Oscar Acceptance Speeches")
plt.show()



In [ ]:
# Step 7: Conclusion & Interpretation
"""
- The bar chart shows which genres have dominated the Best Picture category over time.
- The scatter plot identifies any correlation between box office revenue and audience reception (IMDb ratings).
- The word cloud highlights common themes in Oscar speeches, reflecting industry trends and sentiments. (stretch goal)
"""



### Overview of the Analysis (examples)
- In this analysis, we explored the relationship between the race of law enforcement officers and the race of the drivers they stop. Our goal was to see if there’s any indication of bias in traffic stops based on the racial identity of the officers. To do this, we used a chi-squared test for independence, which helps us understand whether there’s a meaningful connection between these two groups.

### Results of the Chi-Squared Test
- **Chi-Squared Statistic:** We calculated a chi-squared statistic of 122.92. This high number shows that there’s a significant difference between the actual number of stops for different racial groups and what we would expect to see if there were no connection between the officer's race and the driver's race. In other words, this suggests that the patterns we observe in the data are unlikely to be just a coincidence.

- **P-Value:** The p-value we found was about 8.20e-17, which is extremely low. This tells us that the result is statistically significant since it’s much lower than the usual thresholds (like 0.05 or 0.01). A low p-value means we have strong evidence against the idea that there’s no connection between the officer's race and the driver's race.

### Interpretation of Findings
- The results show a strong connection between the race of the officer and the race of the driver being stopped. This means that a driver's chances of being stopped may change depending on the officer's race, suggesting there might be some bias in how traffic stops are carried out.

### Implications
- These findings are important for understanding how race plays a role in law enforcement. They suggest that different racial groups might be treated differently by officers during traffic stops. It's crucial to address these biases to ensure fairness and equality in policing.

### Conclusion
- The strong evidence from the chi-squared statistic and p-value emphasizes the importance of further examining law enforcement practices. Police leaders and community advocacy groups should take these findings into account when reviewing policies and training programs designed to reduce racial bias in policing.